# 11장. 심층 신경망 훈련하기

https://nbviewer.jupyter.org/github/rickiepark/handson-ml2/tree/master/

이 코드의 내용은 Hands-On Machine Learning with Scikit-Learn & TensorFlow을 참고했음을 밝힙니다.

In [ ]:
import tensorflow as tf
from tensorflow import keras

# 글로럿과 He 초기화

저자들은 적절한 신호가 흐르기 위해서는 각 층의 출력에 대한 분산이 입력에 대한 분산과 같아야 한다고 주장합니다. 그리고 역방향에서 층을 통과하기 전과 후의 그레디언트 분산이 동일해야 합니다.

<img src='https://i.imgur.com/iBXgsbj.png' width='100%'>
(층의 입력과 출력 연결 개수를 fan-in과 fan-out이라고 부릅니다.)


<img src='https://i.imgur.com/57ifvaS.png' width='100%'>

In [ ]:
# kernel_initializer='he_uniform' or kernel_initializer='he_normal'

keras.layers.Dense(10, activation='relu', kernel_initializer='he_normal')

In [ ]:
# fan(in) 대신 fan(out) 기반의 균등분포 He 초기화

init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg',
                                          distribution='uniform')
keras.layers.Dense(10, activation="relu", kernel_initializer=init)

# 수렴하지 않는 활성화 함수

### ReLU<br>
* 특정 양숫값에 수렴하지 않는다는 큰 장점
* 죽은 ReLU(dying ReLU): 훈련하는 동안 일부 뉴런이 0 이외의 값을 출력하지 않는다는 의미에서 죽었다고 말합니다. 어떤 경우에는, 특히 큰 학습률을 사용하면 신경망의 뉴런 절반이 죽어 있기도 합니다. 뉴런의 가중치가 바뀌어 훈련 세트에 있는 모든 샘플에 대해 입력의 가중치 합이 음수가 되면 뉴런이 죽게 됩니다. 가중치 합이 음수이면 ReLU 함수의 그레디언트가 0이 되므로 경사 하강법이 더는 작동하지 않습니다.

### LeakyReLU
* LeakyReLU(a, z) = max(az, z)
* 하이퍼파라미터 a가 이 함수가 '새는(leaky)' 정도를 결정합니다. 새는 정도란 z<0일 때 이 함수의 기울기이며, 일반적으로 0.01로 설정합니다. 이 작은 기울기가 LeakyReLU를 절대 죽지 않게 만들어줍니다.
* 최근 한 논문에서 여러 ReLU 함수의 변종을 비교해 얻은 결론 하나는 LeakyReLU가 ReLU보다 항상 성능이 높다는 것입니다.
* 사실 a=0.2(많이 통과)로 하는 것이 a=0.01(조금 통과)보다 더 나은 성능을 내는 것으로 보입니다.


<img src='https://i.imgur.com/Ri6es6i.png' width='100%'>

### RReLU(randomized leaky ReLU)
* 훈련하는 동안 주어진 범위에서 a를 무작위로 선택하고 테스트시에는 평균을 사용

### PReLU(parametric leaky ReLU)
* a가 훈련하는 동안 학습
* 즉, 하이퍼파라미터가 아니고 다른 모델 파라미터와 마찬가지로 역전파에 의해 변경
* 대규모 이미지 데이터셋에서는 ReLU보다 성능이 크게 앞섰지만, 소규모 데이터셋에서는 훈련 세트에 과대적합될 위험 존재

### ELU(exponential linear unit)

*discussion*<br>
z < 0일 때 음숫값이 들어오므로 활성화 함수의 평균 출력이 0에 더 가까워진다?

* z < 0일 때 음숫값이 들어오므로 활성화 함수의 평균 출력이 0에 더 가까워집니다. 이는 앞서 이야기한 그레디언트 소실 문제를 완화해줍니다. 하이퍼파라미터 a는 z가 큰 음숫값일 때 ELU가 수렴할 값을 정의합니다. 보통 1로 설정하지만 다른 하이퍼파라미터처럼 변경할 수 있습니다.
* z < 0이어도 그레디언트가 0이 아니므로 죽은 뉴런을 만들지 않습니다.
* a = 1이면 이 함수는 z = 0에서 급격히 변동하지 않으므로 z = 0을 포함해 모든 구간에서 매끄러워 경사 하강법의 속도를 높여줍니다.
* ELU 활성화 함수의 주요 단점은 (지수 함수를 사용하므로) ReLU나 그 변종들보다 계산이 느리다는 것입니다. 훈련하는 동안에는 수렴 속도가 빨라서 느린 계산이 상쇄되지만 테스트 시에는 ELU를 사용한 네트워크가 ReLU를 사용한 네트워크보다 느릴 것입니다.

<img src='https://i.imgur.com/qAMjuxa.png' width='100%'>

<img src='https://i.imgur.com/qnO3YVa.png' width='100%'>

### SELU(Scaled ELU)
* 저자들은 완전 연결 층만 쌓아서 신경망을 만들고 모든 은닉층이 SELU 활성화 함수를 사용한다면 네트워크가 자기 정규화(self-normalize)된다는 것을 보였습니다.
* 훈련하는 동안 각 층의 출력이 평균 0과 표준편차 1을 유지하는 경향이 있습니다. 이는 그레디언트 소실과 폭주 문제를 막아줍니다.
* 그 결과로 SELU 활성화 함수는 이런 종류의 네트워크(특히 아주 깊은 네트워크)에서 다른 활성화 함수보다 뛰어난 성능을 종종 냅니다.
---
* 입력 특성이 반드시 표준화(평균 0, 표준편차 1)되어야 합니다.
* 모든 은닉층의 가중치는 르쿤 정규분포 초기화로 초기화되어야 합니다. 케라스에서는 kernel\_initializer='lecum\_normal'로 설정합니다.
* 네트워크는 일렬로 쌓은 층으로 구성되어야 합니다. 순환 신경망이나 스킵 연결(skip connection, 즉 와이드 & 딥 네트워크에서 건너뛰어 연결된 층)과 같은 순차적이지 않은 구조에 SELU를 사용하면 자기 정규화되는 것이 보장되지 않습니다. 따라서 SELU가 다른 활성화 함수보다 성능이 뛰어나지 않을 것입니다.

<img src='https://i.imgur.com/OASEpRa.png' width='100%'>

*tip*<br>
* 일반적: SELU > ELU > LeakyReLU(그리고 변종들) > ReLU > tanh > 로지스틱
* 네트워크가 자기 정규화되지 못하는 구조라면 ELU가 SELU보다 성능이 더 나을 수 있습니다.
* 실행 속도가 중요하다면 LeakyReLU를 선택할 수 있습니다. 하이퍼파라미터를 더 추가하고 싶지 않다면 케라스에서 사용하는 기본값 a를 사용합니다(예를 들어, LeakyReLU는 0.3).
* 시간과 컴퓨팅 파워가 충분하다면 교차 검증을 사용해 여러 활성화 함수를 평가해볼 수 있습니다.
* 신경망이 과대적합되었다면 RReLU
* 훈련 세트가 아주 크다면 PReLU
* ReLU가 (지금까지) 가장 널리 사용되는 활성화 함수이므로 많은 라이브러리와 하드웨어 가속기들은 ReLU에 특화되어 최적화되어 있습니다. 따라서 속도가 중요하다면 ReLU가 가장 좋은 선택일 것입니다.

In [ ]:
# he_normal
model = keras.models.Sequential([
    pass
    keras.layers.Dense(10, kernel_initializer='he_normal'),
    keras.layers.LeakyReLU(alpha=0.2)
])

# selu
layer = keras.layer.Dense(10, activation='selu', kernel_initializer='lecun_normal')

# 배치 정규화

* 각 층에서 활성화 함수를 통과하기 전에나 후에 모델링 연산을 하나 추가
* 이 연산은 단순하게 입력을 원점에 맞추고 정규화한 다음, 각 층에서 두 개의 새로운 파라미터(하나는 스케일 조정에, 다른 하나는 이동에 사용)로 결괏값의 스케일을 조정하고 이동
* 많은 경우 신경망의 첫 번째 층으로 배치 정규화를 추가하면 훈련 세트를 (예를 들면 StandardScaler를 사용하여) 표준화할 필요가 없습니다. 배치 정규화 층이 이런 역할을 대신합니다.
* 그레디언트 소실 문제가 크게 감소하여 하이퍼볼릭 탄젠트나 로지스틱 활성화 함수 같은 수렴성을 가진 활성화 함수를 사용할 수 있습니다.
* 가중치 초기화에 네트워크가 훨씬 덜 민감해집니다.
* 저자들은 훨씬 큰 학습률을 사용하여 학습 과정의 속도를 크게 높일 수 있었습니다.
* 배치 정규화는 규제와 같은 역할을 하여 다른 규제 기법의 필요성을 줄여줍니다.
* 배치 정규화를 사용할 때 에포크마다 더 많은 시간이 걸리므로 훈련이 오히려 느려질 수 있습니다. 하지만 배치 정규화를 사용하면 수렴이 훨씬 빨라지므로 보통 상쇄됩니다. 따라서 더 적은 에포크로 동일한 성능에 도달할 수 있습니다.
---
* 모델의 복잡도를 키웁니다.
* 실행 시간 면에서도 손해입니다. 층마다 추가되는 계산이 신경망의 예측을 느리게 합니다.

<img src='https://i.imgur.com/fxF9DRt.png' width='100%'>

* $\gamma$(출력 스케일 벡터)와 $\beta$(출력 이동 벡터)는 일반적인 역전파를 통해 학습됩니다.
* $\mu$(최종 입력 평균 벡터)와 $\sigma$(최종 입력 표준편차 벡터)는 지수 이동 평균을 사용하여 추정됩니다.
* $\mu$와 $\sigma$는 훈련하는 동안 추정되지만 훈련이 끝난 후에 사용됩니다.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

* 배치 정규화 층은 입력마다 네 개의 파라미터 $\gamma$, $\beta$, $\mu$, $\sigma$를 추가합니다(예를 들면 첫 번째 배치 정규화 층은 4x784=3136개의 파라미터가 있습니다).
* 마지막 두 개의 파라미터 $\mu$와 $\sigma$는 이동 평균입니다. 이 파라미터는 역전파로 학습되지 않기 때문에 케라스는 'Non-trainable' 파라미터로 분류합니다(배치 정규화 파라미터의 전체 개수는 3136+1200+400입니다. 이를 2로 나누면 이 모델에서 훈련되지 않는 전체 파라미터 개수 2386을 얻습니다).
* 배치 정규화 논문의 저자들은 활성화 함수 이후보다 활성화 함수 이전에 배치 정규화 층을 추가하는 것이 좋다고 조언합니다. 하지만 작업에 따라 선호되는 방식이 달라서 이 조언에 대해서는 논란이 조금 있습니다.

In [ ]:
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

* 활성화 함수 전에 배치 정규화 층을 추가하려면 은닉층에서 활성화 함수를 지정하지 말고 배치 정규화 층 뒤에 별도의 층으로 추가해야 합니다.
* 배치 정규화 층은 입력마다 이동 파라미터를 포함하기 때문에 이전 층에서 편향을 뺄 수 있습니다(층을 만들 때 use_bias=False로 설정합니다).

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

BatchNormalization 클래스는 조정할 하이퍼파라미터가 적습니다. 보통 기본값이 잘 동작하지만 momentum 매개변수를 변경해야 할 수 있습니다. BatchNormalization 층이 지수 이동 평균을 업데이트할 때 이 하이퍼파라미터를 사용합니다. 새로운 값 v(즉, 현재 배치에서 계산한 새로운 입력 평균 벡터나 표준편차 벡터)가 주어지면 다음 식을 사용해 이동 평균 v_hat을 업데이트합니다. 적절한 모멘텀 값은 일반적으로 1에 가깝습니다. 예를 들면 0.9, 0.99, 0.999입니다(데이터셋이 크고 미니배치가 작으면 소수점 뒤에 9를 더 넣어 1에 가깝게 합니다).

<img src='https://i.imgur.com/OFPH9Du.png' width='100%'>

# 그레디언트 클리핑

그레디언트 폭주 문제를 완화하는 인기 있는 다른 방법은 역전파될 때 일정 임계값을 넘어서지 못하게 그레디언트를 잘라내는 것입니다. 이를 그레디언트 클리핑(gradient clipping)이라고 합니다. 순환 신경망은 배치 정규화를 적용하기 어려워서 이 방법을 많이 사용합니다. 다른 종류의 네트워크는 배치 정규화면 충분합니다.

In [ ]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.complie(loss='mse', optimizer=optimizer)

이 옵티마이저는 그레디언트 벡터의 모든 원소를 -1.0과 1.0 사이로 클리핑합니다. 즉 (훈련되는 각 파라미터에 대한) 손실의 모든 편미분 값을 -1.0에서 1.0으로 잘라냅니다. 임계값은 하이퍼파라미터로 튜닝할 수 있습니다. 이 기능은 그레디언트 벡터의 방향을 바꿀 수 있습니다. 예를 들어 원래 그레디언트 벡터가 [0.9, 100.0]라면 대부분 두 번째 축 방향을 향합니다. 하지만 값을 기준으로 이를 클리핑을 하면 [0.9, 1.0]이 되고 거의 두 축 사이 대각선 방향을 향합니다. 실전에서는 이 방식이 잘 동작합니다. 만약 그레디언트 클리핑이 그레디언트 벡터의 방향을 바꾸지 못하게 하려면 clipvalue 대신 clipnorm을 지정하여 노름으로 클리핑해야 합니다. 만약 L2 노름이 지정한 임계값보다 크면 전체 그레디언트를 클리핑합니다. 예를 들어 clipnorm=1.0으로 지정한다면 벡터 [0.9, 100.0]이 [0.00899964, 0.9999595]로 클리핑되므로 방향을 그대로 유지합니다. 하지만 첫 번째 원소는 거의 무시됩니다. 훈련하는동안 그레디언트가 폭주한다면 다른 임계값으로 값과 노름을 모두 사용하여 클리핑할 수 있습니다(텐서보드를 사용해 그레디언트의 크기를 추적할 수 있습니다). 그리고 검증 세트에서 어떤 방식이 가장 좋은 성과를 내는지 확인할 수 있습니다.

*discussion*<br>
clipvalue를 사용하면 벡터의 방향이 바뀌는데 실전에서 잘 동작하는 이유?

# 사전훈련된 층 재사용하기

전이 학습(transfer learning): 해결하려는 것과 비슷한 유형의 문제를 처리한 신경망이 이미 있는지 찾아본 다음, 그 신경망의 하위층을 재사용하는 방법. 이 방법은 훈련 속도를 크게 높일 뿐 아니라 필요한 훈련 데이터도 크게 줄여줍니다.

<img src='https://i.imgur.com/XOdtk3h.png' width='100%'>

* 만약 원래 문제에서 사용한 것과 크기가 다른 이미지를 입력으로 사용한다면 원본 모델에 맞는 크기로 변경하는 전처리 단계를 추가해야 합니다. 일반적으로 전이 학습은 저수준 특성이 비슷한 입력에서 잘 작동합니다.
* 보통 원본 모델의 출력층을 바꿔야 합니다. 이 층이 새로운 작업에 각장 유용하지 않는 층이고 새로운 작업에 필요한 출력 개수와 맞지 않을 수도 있습니다.
* 비슷하게 원본 모델의 상위 은닉층은 하위 은닉층보다 덜 유용합니다. 새로운 작업에 유용한 고수준 특성은 원본 작업에서 유용했던 특성과는 상당히 다르기 때문입니다. 재사용할 층 개수를 잘 선정하는 것이 필요합니다.
* 작업이 비슷할수록 (낮은 층부터 시작해서) 더 많은 층을 재사용하세요. 아주 비슷한 작업이라면 모든 은닉층을 유지하고 출력층만 교체합니다.
* 먼저 재사용하는 층을 모두 동결합니다(즉, 경사 하강법으로 가중치가 바뀌지 않도록 훈련되지 않는 가중치로 만듭니다). 그다음 모델을 훈련하고 성능을 평가합니다. 맨 위에 있는 한두개의 은닉층의 동결을 해제하고 역전파를 통해 가중치를 조정하여 성능이 향상되는지 확인합니다. 훈련 데이터가 많을수록 많은 층의 동결을 해제할 수 있습니다. 재사옹 층의 동결을 해제할 때는 학습률을 줄이는 것이 좋습니다. 가중치를 세밀하게 튜닝하는 데 도움이 됩니다.
* 만약 여전히 좋은 성능을 낼 수 없고 훈련 데이터가 적다면 상위 은닉층(들)을 제거하고 남은 은닉층을 다시 동결해보세요. 이런 식으로 재사용할 은닉층의 적절한 개수를 찾을 때까지 반복합니다. 훈련 데이터가 아주 많다면 은닉층을 제거하는 대신 다른 것으로 바꾸거나 심지어 더 많은 은닉층을 추가할 수도 있습니다.

In [ ]:
# model_A: 8개의 클래스가 담겨 있는 패션 MNIST 데이터셋을 분류하는 모델
# model_B_on_A: 셔츠와 샌들만을 분류하는 이진 분류기(패션 MNIST에 존재하는 클래스)

model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))
model_A = keras.models.load_model("my_model_A.h5")  # 출력층만 제외하고 모든 층을 재사용
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])

model\_A와 model\_B\_on\_A는 일부 층을 공유합니다. model\_B\_on\_A를 훈련할 때 model\_A도 영향을 받습니다. 이를 원치 않는다면 층을 재사용하기 전에 model\_A를 클론하세요. clone\_moodel() 메서드로 모델 A의 구조를 복제한 후 가중치를 복사합니다(clone\_model() 메서드는 가중치를 복제하지 않습니다).

In [ ]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

새로운 출력층이 랜덤하게 초기화되어 있으므로 큰 오차를 만들 것입니다(적어도 처음 몇 번의 에포크 동안). 따라서 큰 오차 그레디언트가 재사용된 가중치를 망칠 수 있습니다. 이를 피하는 한 가지 방법은 처음 몇 번의 에포크 동안 재사용된 층을 동결하고 새로운 층에게 적절한 가중치를 학습할 시간을 주는 것입니다. 이를 위해 모든 층의 trainable 속성을 False로 지정하고 모델을 컴파일합니다.

*Note*<br>
compile() 메서드가 모델에서 훈련될 가중치를 모으기 때문에 동결을 하거나 해제한 후에는 반드시 다시 컴파일해야 합니다.

이제 몇 번의 에포크 동안 모델을 훈련할 수 있습니다. 그다음 재사용된 층의 동결을 해제하고 (모델을 다시 컴파일해야 합니다) 작업 B에 맞게 재사용된 층을 세밀하게 튜닝하기 위해 훈련을 계속합니다. 일반적으로 재사용된 층의 동결을 해제한 후에 학습률을 낮추는 것이 좋습니다. 이렇게 하면 재사용된 가중치가 망가지는 것을 막아줍니다.

In [ ]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-4),   # 기본 학습률은 1e-2
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

In [ ]:
model_B_on_A.evaluate(X_test_B, y_test_B)

* 전이 학습은 조금 더 일반적인 특성을 (특히 아래쪽 층에서) 감지하는 경향이 있는 심층 합성곱 신경망에서 잘 동작합니다.

# 비지도 사전훈련

* 레이블되지 않은 훈련 데이터를 많이 모을 수 있다면 이를 사용하여 오토인코더(autoencoder)나 생성적 적대 신경망과 같은 비지도 학습 모델을 훈련할 수 있습니다. 그다음 오토인코더나 GAN 판별자의 하위층을 재사용하고 그 위에 새로운 작업에 맞는 출력층을 추가할 수 있습니다. 그다음 지도 학습으로 (즉, 레이블된 훈련 샘플로) 최종 네트워크를 세밀하게 튜닝합니다.
* 딥러닝 초기에는 층이 많은 모델을 훈련하는 것이 어려웠기 때문에 탐욕적 층 단위 사전훈련(greedy layer-wise pretraining)이라고 부르는 기법을 사용했습니다. 먼저 하나의 층을 갖긴 비지도 학습 모델을 훈련합니다. 일반적으로 RBM을 사용합니다. 그다음 이 층을 동결하고 그 위에 다른 층을 추가한 다음 모델을 다시 훈련합니다(새로 추가한 층만 훈련하기 위해서). 그다음 새로운 층을 동결하고 그 위에 또 다른 층을 추가하고 모델을 다시 훈련하는 식으로 반복됩니다. 오늘날에는 훨씬 간단한 방법을 사용합니다. 일반적으로 한 번에 전체 비지도 학습 모델을 훈련하고 RBM 대신 오토인코더나 GAN을 사용합니다.
* 비지도 훈련에서는 비지도 학습 기법으로 레이블이 없는 데이터(또는 전체 데이터)로 모델을 훈련합니다. 그다음 지도 학습 기법을 사용하여 레이블된 데이터에서 최종 학습을 위해 세밀하게 튜닝합니다. 비지도 학습 부분은 한 번에 하나의 층씩 훈련하거나 바로 전체 모델을 훈련할 수도 있습니다.

<img src='https://i.imgur.com/26lQlSJ.png' width='100%'>

# 보조 작업에서 사전훈련

레이블된 훈련 데이터가 많지 않다면 마지막 선택 사항은 레이블된 훈련 데이터를 쉽게 얻거나 생성할 수 있는 보조 작업에서 첫 번째 신경망을 훈련하는 것입니다. 그리고 이 신경망의 하위 층을 실제 작업을 위해 재사용합니다. 첫 번째 신경망의 하위 층은 두 번째 신경망에 재사용될 수 있는 특성 추출기를 학습하게 됩니다.<br>
예를 들어 얼굴을 인식하는 시스템을 만들려고 하는데 개인별 이미지가 얼마 없다면 좋은 분류기를 훈련하기에 충분하지 않습니다. 각 사람의 사진을 수백 개씩 모으기는 현실적으로 어렵습니다. 그러나 인터넷에서 무작위로 많은 인물의 이미지를 수집해서 두 개의 다른 이미지가 같은 사람의 것인지 감지하는 첫 번째 신경망을 훈련할 수 있습니다. 이런 신경망은 얼굴의 특성을 잘 감지하도록 학습될 것입니다. 그러므로 이런 신경망의 하위층을 재사용해 적은 양의 훈련 데이터에서 얼굴을 잘 구분하는 분류기를 훈련할 수 있습니다.

# 고속 옵티마이저

훈련 속도를 높이는 방법

* 연결 가중치에 좋은 초기화 전략 사용하기
* 좋은 활성화 함수 사용하기
* 배치 정규화 사용하기
* (보조 작업 또는 비지도 학습을 사용하여 만들 수 있는) 사전훈련된 네트워크의 일부 재사용하기
* 고속 옵티마이저 사용하기

### 모멘텀 최적화

In [ ]:
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9)

<img src='https://i.imgur.com/U8GKXL8.png' width='100%'>

볼링공이 매끈한 표면의 완만한 경사를 따라 굴러간다고 합시다. 처음에는 느리게 출발하지만 종단속도에 도달할 때까지는 빠르게 가속될 것입니다(마찰이나 공기 저항이 없다면). 이것이 모멘텀 최적화의 간단한 원리입니다.<br>
* 모멘텀 최적화는 이전 그레디언트가 얼마였는지를 상당히 중요하게 생각합니다.
* 입력값의 스케일이 매우 다른 경우에도 모멘텀 최적화는 골짜기를 따라 바닥(최적점)에 도달할 때까지 점점 더 빠르게 내려갑니다.
* 지역 최적점(local optima)을 건너뛰도록 하는 데도 도움이 됩니다.
* 모멘텀 떄문에 옵티마이저가 최적값에 안정되기 전까지 건너뛰었다가 다시 돌아오고, 다시 또 건너뛰는 식으로 여러 번 왔다 갔다 할 수 있습니다. 이것이 시스템에 마찰 저항이 조금 있는 것이 좋은 이유입니다. 이는 이런 진동을 없애주고 빠르게 수렴되도록 합니다.
* 모멘텀 최적화의 한 가지 단점은 튜닝할 하이퍼파라미터가 하나 늘어난다는 것입니다. 그러나 실제로 모멘텀 0.9에서 보통 잘 작동하며 경사 하강법보다 거의 항상 더 빠릅니다.

### 네스테로프 가속 경사

In [ ]:
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)

<img src='https://i.imgur.com/Mf0f49U.png' width='100%'>

네스테로프 가속 경사는 현재 위치가 $\theta$가 아니라 모멘텀의 방향으로 조금 앞선 $\theta+\beta m$에서 비용 함수의 그레디언트를 계산하는 것입니다. 일반적으로 모멘텀 벡터가 올바른 방향(즉, 최적점을 향하는 방향)을 가리킬 것이므로 이런 변경이 가능합니다.

<img src='https://i.imgur.com/a6OrvYe.png' width='100%'>

### AdaGrad

In [ ]:
optimizer = keras.optimizers.Adagrad(learning_rate=0.001)

경사 하강법은 전역 최적점 방향으로 곧장 향하지 않고 가장 가파른 경사를 따라 빠르게 내려가기 시작해서 골짜기 아래로 느리게 이동합니다. 알고리즘이 이를 일찍 감지하고 전역 최적점 쪽으로 좀 더 정확한 방향을 잡았다면 좋았을 것입니다. AdaGrad 알고리즘은 가장 가파른 차원을 따라 그레디언트 벡터의 스케일을 감소시켜 이 문제를 해결합니다.
* 이 알고리즘은 학습률을 감소시키지만 경사가 완만한 차원보다 가파른 차원에 대해 더 빠르게 감소됩니다. 이를 적응적 학습률(adaptive learning rate)이라고 부르며, 전역 최적점 방향으로 더 곧장 가도록 갱신되는 데 도움이 됩니다.
* 학습률 파라미터 $\eta$를 덜 튜닝해도 되는 점이 또 하나의 장점입니다.
* AdaGrad는 간단한 2차방정식 문제에 대해서는 잘 작동하지만 신경망을 훈련할 때 너무 일찍 멈추는 경우가 종종 있습니다. 학습률이 너무 감소되어 전역 최적점에 도착하기 전에 알고리즘이 완전히 멈춥니다. 그래서 케라스에 AdaGrad 옵티마이저가 있지만 심층 신경망에는 사용하지 말아야 합니다(하지만 선형 회귀 같은 간단한 작업에는 효과적일 수 있습니다). 하지만 AdaGrad를 알면 다른 적응적 학습률 옵티마이저를 이해하는 데 도움이 됩니다.

<img src='https://i.imgur.com/Y9MTaDj.png' width='100%'>

<img src='https://i.imgur.com/ieYHpz8.png' width='100%'>

### RMSProp

In [ ]:
optimizer = keras.optimizers.RMSprroop(lr=0.001, rho=0.9)

AdaGrad는 너무 빨리 느려져서 전역 최적점에 수렴하지 못하는 위험이 있습니다. RMSProp 알고리즘은 (훈련 시작부터의 모든 그레이드가 아닌) 가장 최근 반복에서 비롯된 그레디언트만 누적함으로써 이 문제를 해결했습니다. 이렇게 하기 위해 알고리즘의 첫 번째 단꼐에서 지수 감소를 사용합니다.
* 보통 감쇠율 $\beta$는 0.9로 설정합니다.
* 아주 간단한 문제를 제외하고는 이 옵티마이저각 언제나 AdaGrad보다 훨씬 더 성능이 좋습니다.

<img src='https://i.imgur.com/iPRZ4IT.png' width='100%'>

### Adam

In [ ]:
optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.009)

적응적 모멘트 추정(adaptive moment estimation)을 의미하는 Adam은 모멘텀 최적화와 RMSProp의 아이디어를 합친 것입니다. 모멘텀 최적화처럼 지난 그레디언트의 지수 감소 평균을 따르곡 RRMSPrrop처럼 지난 그레디언트 제곱의 지수 감소된 평균을 따릅니다.
* 모멘텀 감쇠 하이퍼파라미터 $\beta_1$은 보통 0.9로 초기화하고 스케일 감쇠 하이퍼파라미터 $\beta_2$는 0.999로 초기화하는 경우가 많습니다.
* Adam이 (AdaGrad나 RMSProp처럼) 적응적 학습률 알고리즘이기 때문에 학습률 하이퍼파라미터 $\eta$를 튜닝할 필요가 적습니다. 기본값 $\eta=0.001$을 일반적으로 사용하므로 경사 하강법보다 Adam이 사용하기 더 쉽습니다.

<img src='https://i.imgur.com/3kEPggm.png' width='100%'>

### AdaMax

### Nadam

# 학습률 스케줄링

<img src='https://i.imgur.com/gFvuipN.png' width='100%'>

일정한 학습률보다 더 나은 방법이 있습니다. 큰 학습률로 시작하고 학습 속도가 느려질 때 핛습률을 낮추면 최적의 고정 학습률보다 좋은 솔루션을 더 빨리 발견할 수 있습니다. 훈련하는 동안 학습률을 감소시키는 전략에는 여러 가지가 있습니다. 이런 전략을 학습 스케줄(learning schedule)이라고 합니다.

* 거듭제곱 기반 스케줄링(power scheduling)<br>
학습률을 반복 횟수 t에 대한 함수 $\eta (t)=\eta_0 / (1+t/s)^c$로 지정합니다. 초기 학습률 $\eta_0$, 거듭제곱 수 c(일반적으로 1로 지정합니다), 스텝 횟수 s는 하이퍼파라미터입니다. 학습률은 각 스텝마다 감소합니다. s번 스텝 뒤에 학습률은 $\eta_0/2$으로 줄어듭니다. s번 더 스텝이 진행된 후 학습률은 $\eta_0/3$으로 줄어들고 그다음 $\eta_0/4$으로 줄어들고 그다음 $\eta_0/5$이 되는 식입니다. 여기서 볼 수 있듯이 처음에는 빠르게 감소하다가 점점 더 느리게 감소됩니다. 물론 거듭제곱 기반 스케줄링은 $\eta_0$과 s를 (아마 c도) 튜닝해야 합니다.

* 지수 기반 스케줄링(exponential scheduling)<br>
핛습률을 $\eta (t)=\eta_0 0.1^{t/s}$로 설정합니다. 학습률이 s 스텝마다 10배씩 점차 줄어들 것입니다. 거듭제곱 기반 스케줄링이 학습률을 갈수록 천천히 감소시키는 반면 지수 기반 스케줄링은 s번 스텝마다 계속 10배씩 감소합니다.

* 구간별 고정 스케줄링(piecewise constant scheduling)<br>
일정 횟수의 에포크 동안 일정한 학습률을 사용하고 (예를 들어, 5에포크 동안 $\eta_0=0.1$) 그다음 또 다른 횟수의 에포크 동안 작은 학습률을 사용하는 (예를 들어, 50에포크 동안 $\eta_1=0.001$) 식입니다. 이 방법이 잘 동작할 수 있지만 적절한 학습률과 에포크 횟수의 조합을 찾으려면 이리저리 바꿔봐야 합니다.

* 성능 기반 스케줄링(performance scheduling)<br>
매 N 스텝마다 (조기 종료처럼) 검증 오차를 측정하고 오차가 줄어들지 않으면 $\lambda$배만큼 학습률을 감소시킵니다.

* 1사이클 스케줄링(1cycle scheduling)<br>
...skip...

In [ ]:
# 거듭제곱 기반 스케줄링

optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-4)   # decay: s의 역수

In [ ]:
# 지수 기반 스케줄링

def exponential_decay_fn(epoch):
    return 0.01 * 0.1 ** (epoch/20)

def expoonential_decay(lr0, s):
    def exponential_decay_fn(epooch):
        return lr0 * 0.1 ** (epoch/s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0=0.01, s=20)

# def exponential_decay_fn(epoch, lr):
#     return lr * 0.1 ** (1/20)

In [ ]:
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid),
                    callbacks=[lr_scheduler])

In [ ]:
# 구간별 고정 스케줄링

def piecewise_constant_fn(epoch):
    if epoch < 5:
        return 0.01
    elif epoch < 15:
        return 0.005
    else:
        return 0.001

In [ ]:
# 성능 기반 스케줄링

lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

# L1과 L2 규제

In [ ]:
# keras.regularizers.l1(0.01)
# keras.regularizers.l2(0.01)
# keras.regularizers.l1_l2(l1=0.01, l2=0.01)

layer = keras.layer.Dense(100, activation='elu',
                          kernel_initializer='he_normal',
                          kernel_regularizer=keras.regularizers.l2(0.01))

신경망의 연결 가중치를 제한하기 위해 l2규제를 사용하거나 (많은 가중치가 0인) 희소 모델을 만들기 위해 l1 규제를 사용할 수 있습니다.

# 코드 리팩터링(refactoring)

In [ ]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation='elu',
                           kernel_initializer='he_normal',
                           kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
                                 keras.layers.Flatten(input_shape=[28, 28]),
                                 RegularizedDense(300),
                                 RegularizedDense(100),
                                 RegularizedDense(10, activation='softmax',
                                                  kernel_initializer='glorot_uniform')
])

# 드롭아웃

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

어떤 회사에서 직원들이 아침마다 출근할지 말지 동전 던지기로 결정한다면 회사의 일이 더 잘될까요? 글쎄요, 누가 알겠어요. 어쩌면 정말 그럴지도 모르죠! 회사가 이런 식으로 운영된다면 커피머신의 원두를 채우는 일이나 아주 중요한 어떤 업무도 한 사람에게 전적으로 의지할 수 없을 것이고, 전문성이 여러 사람에게 나뉘어져 있어야 합니다. 직원들은 주변 동료뿐만 아니라 많은 다른 직원과 협력하는 법을 배워야 합니다. 이 회사는 유연성이 훨씬 높아질 것입니다. 한 직원이 직장을 떠나도 크게 달라지는 것이 없을 것입니다.

매 훈련 스텝에서 각 뉴런(입력 뉴런은 포함하고 출력 뉴런은 제외합니다)은 임시적으로 드롭아웃될 확률 p를 가집니다. 즉, 이번 훈련 스텝에는 완전히 무시되지만 다음 스텝에서는 활성화될 수 있습니다. 하이퍼파라미터 p를 드롭아웃 비율(dropout rate)이라고 하고 보통 10%와 50% 사이를 지정합니다. 순환 신경망에서는 20\~30%에 가깝고 합성곱 신경망에서는 40\~50%에 가깝습니다. 훈련이 끝난 후에는 뉴런에 더는 드롭아웃을 지정하지 않습니다.
* 일반적으로 (출력층을 제외한) 맨 위의 층부터 세 번째 층까지 있는 뉴런에만 드롭아웃을 적용합니다.
* 훈련이 끝난 뒤 각 입력의 연결 가중치에 보존 확률(keep probability) (1-p)를 곱해야 합니다. 또는 훈련하는 동안 각 뉴런의 출력을 보존 확률로 나눌 수도 있습니다.
* 드롭아웃은 훈련하는 동안에만 활성화되므로 훈련 손실과 검증 손실을 비교하면 오해를 일으키기 쉽습니다. 특히 비슷한 훈련 손실과 검증 손실을 얻었더라도 모델이 훈련 세트에 과대적합될 수 있습니다. 따라서 (예를 들어 훈련이 끝난 후) 드롭아웃을 빼고 훈련 손실을 평가해야 합니다.
* 모델이 과대적합되었다면 드롭아웃 비율을 늘릴 수 있습니다. 반대로 모델이 훈련 세트에 과소적합되면 드롭아웃 비율을 낮추어야 합니다. 층이 클 때는 드롭아웃 비율을 늘리고 작은 층에는 드롭아웃 비율을 낮추는 것이 도움이 됩니다. 또한 많은 최신의 신경망 구조는 마지막 은닉층 뒤에만 드롭아웃을 사용합니다.

<img src='https://i.imgur.com/UpG6TRq.png' width='100%'>

# 몬테 카를로 드롭아웃

드롭아웃으로 만든 예측을 평균하면 일반적으로 드롭아웃이 없이 예측한 하나의 결과보다 더 안정적입니다.

In [ ]:
y_probas = np.stack([model(X_test_scaled, training=True)    # training=True: 드롭아웃 층 활성화
                     for sample in range(100)])
y_proba = y_probas.mean(axis=0)
y_std = y_probas.std(axis=0)

# 맥스-노름 규제

각각의 뉴런에 대해 입력의 연결 가중치 w가 $||w||_2\leq r$이 되도록 제한합니다. r은 맥스-노름 하이퍼파라미터이고 ||.||은 l2 노름을 나타냅니다.

In [ ]:
keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal',
                   kernel_constraint=keras.constraints.max_norm(1.))

# 요약 및 실용적인 가이드라인

|하이퍼파라미터|기본값|
|---|---|
|커널 초기화|He 초기화|
|활성화 함수|ELU|
|정규화|얕은 신경망일 경우 없음. 깊은 신경망이라면 배치 정규화|
|규제|조기 종료(필요하면 l2 규제 추가)|
|옵티마이져|모멘텀 최적화(또는 RMSProp이나 Nadam)|
|학습률 스케줄|1사이클|


* 입력 특성을 정규화하기
* 비슷한 문제를 해결한 모델을 찾을 수 없다면 사전훈련된 신경망의 일부를 재사용하기
* 레이블이 없는 데이터가 많다면 비지도 사전훈련 사용
* 비슷한 작업을 위한 레이블된 데이터가 많다면 보조 작업에서 사전 훈련 수행
* 희소 모델이 필요하다면 l1 규제 사용(훈련된 후 작은 가중치를 0으로)
* 빠른 응답을 하는 모델(번개처럼 빨리 예측하는 모델)이 필요하면 층 개수를 줄이고 배치 정규화 층을 이전 층에 합쳐라. LeakyReLU나 ReLU와 같이 빠른 활성화 함수를 사용해라. 희소 모델을 만들어라. 부동소수점 정밀도를 32비트에서 16비트 혹은 8비트로 낮춰라.
* 위험에 민감하고 예측 속도가 매우 중요하지 않은 애플리케이션이라면 성능을 올리고 불확실성 추정과 신뢰할 수 있는 확률 추정을 얻기 위해 MC 드롭아웃을 사용하라.